In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None #allows writing upon temporary DFs
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

pwd_postgresql= #***enter password here***

In [ ]:
#read in the kaggle file 
kgldf = pd.read_csv("all_weekly_excess_deaths.csv")

In [ ]:
# keep only the (potentially) relevant info
kgl= kgldf[['country', 'start_date', 'end_date', 'week', 'population', 'excess_deaths', 'excess_deaths_per_100k']]


In [ ]:
# check date data type

print(kgl.dtypes)

In [ ]:
kgl.head()

In [ ]:
# create new variable that has dates in python's date format
kgl['newstartdate']=pd.to_datetime(kgl['start_date'].copy())

In [ ]:
print(kgl.dtypes)

In [ ]:
# do the same for the end dates
kgl['newenddate']=pd.to_datetime(kgl['end_date'])

In [ ]:
# to estimate the period of the final combined dataset, we decided the common start and end dates for countries between datasets 
# we start with identifying the latest date a country begins to provide data 
lateststartdatekgl= kgl.groupby('country')['newstartdate'].min()

In [ ]:
#we see that the latest starting date is Dec 2019
lateststartdatekgl.unique()

In [ ]:
# carry out the same process for the end date: these are the earliest dates a country ends their data output
# we identify the earliest date a country ends their data output
earliestenddatekgl= kgl.groupby('country')['newenddate'].max()

In [ ]:
# change the groupby output to DF format to allow other commands to work on it
earliestenddatekgl.to_frame()

In [ ]:
#this shows that most countries end their data in July 2022
earliestenddatekgl.plot()

In [ ]:
earliestenddatekgl.unique()

In [ ]:
#change 'newstartdate' to 'newdate' to allow merge on this variable with the OWID dataset
kgl.rename(columns={'newstartdate':'newdate'}, inplace=True)

In [ ]:
#drop unnecessary columns
kgl.drop(['start_date', 'end_date', 'week', 'population', 'newenddate'], axis=1, inplace=True)

In [ ]:
#now we read in the 'our world in data (OWID)' file
owiddf = pd.read_csv("owid-covid-data.csv")

In [ ]:
#select relevant columns & variables
owid= owiddf[['location', 'date', 'new_cases', 'population']]

In [ ]:
#repeat the process performed with the kaggle dataset above
#make new date format date variable
owid['newdate']=pd.to_datetime(owid['date'])

In [ ]:
print(owid.dtypes)

In [ ]:
owid['incidence']=owid['new_cases']/owid['population']*100000

In [ ]:
owid

In [ ]:
#change the country info to the same name as the kaggle data
owid.rename(columns={'location':'country'}, inplace=True)

In [ ]:
#get latest 'start of data' date for each country
lateststartdateowid= owid.groupby('country')['newdate'].min()

In [ ]:
#change output to DF format
lateststartdateowid= lateststartdateowid.to_frame()

In [ ]:
#this is later than the kaggle start date, so we use data from 1/3/2020
lateststartdateowid.describe()

In [ ]:
#visual representation showing the spread of data
lateststartdateowid.plot()

In [ ]:
#get earliest 'final data date' of each country
earliestenddateowid= owid.groupby('country')['newdate'].max()

In [ ]:
earliestenddateowid.to_frame()

In [ ]:
earliestenddateowid.describe()

In [ ]:
#this is later than the kaggle data, so we use 31/7/2022 as the last date
earliestenddateowid.plot()


In [ ]:
owidweekly= owid

In [ ]:
print(owid.dtypes)

In [ ]:
owidweekly= owid.groupby('country')['newdate', 'new_cases', 'incidence',].resample('W-Mon', label='right', closed='right', on='newdate').sum().reset_index()

In [ ]:
owidweekly

In [ ]:
#merge on country and dates that fall between 1/3/20 and 31/7/22
mergeddf= owidweekly.merge(kgl, on=['country', 'newdate'])

In [ ]:
mergeddf

__Connecting to the local database__

In [ ]:
protocol = 'postgresql'
username = 'postgres'  
password = pwd_postgresql
host = 'localhost'
port = 5432                
database_name = 'covid_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
#Checking that the table has been created
engine.table_names()

In [ ]:
#Using pandas to load dataframe into database
mergeddf.to_sql(name='covid_deaths', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from covid_deaths', con=engine)